In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('data/cleaned_reviews.csv')
df.head()

,Movie_N,cleaned_review,Stars
0,The Old Guard,love concept execution not bad there some crin...,7
1,The Old Guard,there were just so many clichés cringe naive m...,5
2,The Old Guard,actors all good well thats story childlike dir...,5
3,The Old Guard,my mind kept referring back austin powers badd...,9
4,The Old Guard,london scenes end were just bizarre empty stre...,5


In [5]:
df.dropna(subset=['cleaned_review'], inplace=True)


In [6]:
df['Stars']=df['Stars'].astype(int)

In [8]:
df

,Movie_N,cleaned_review,Stars,label
0,The Old Guard,love concept execution not bad there some crin...,7,2
1,The Old Guard,there were just so many clichés cringe naive m...,5,1
2,The Old Guard,actors all good well thats story childlike dir...,5,1
3,The Old Guard,my mind kept referring back austin powers badd...,9,2
4,The Old Guard,london scenes end were just bizarre empty stre...,5,1
...,...,...,...,...
5676,Greyhound,my only complaint movie could have been longer...,5,1
5677,Greyhound,though movie might have been right technical t...,8,2
5678,Greyhound,i felt movie could have been longer better dep...,1,0
5679,Greyhound,climax if any not up mark,9,2


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import joblib
import xgboost as xgb
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier  # BEGIN:

from sklearn.svm import LinearSVC  # END:

In [10]:
X_text = df["cleaned_review"]
y = df["label"]

In [30]:
vector=joblib.load("models/tfidf_vectorizer.pkl")
X=vector.fit_transform(X_text)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=42)

In [55]:
#{'C': 1, 'max_iter': 500, 'solver': 'lbfgs'}
model = RandomForestClassifier(
    class_weight='balanced',
    n_estimators=100,
    random_state=42
)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [56]:

model.score(X_test, y_test)

0.6388443017656501

In [57]:
arr=model.predict(X_test)
arr

array([2, 2, 2, ..., 2, 2, 2])

In [60]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=["Negative", "Neutral", "Positive"]))


              precision    recall  f1-score   support

    Negative       0.42      0.39      0.41       341
     Neutral       0.12      0.04      0.05       114
    Positive       0.70      0.79      0.74       791

    accuracy                           0.61      1246
   macro avg       0.41      0.41      0.40      1246
weighted avg       0.57      0.61      0.59      1246



In [59]:
review="not good not bad just ok"
def predict_sentiment(review):
    review_vectorized = vector.transform([review])
    prediction = model.predict(review_vectorized)
    x=prediction[0]
    if x==1:
        print("Neutral")
    elif x==0:
        print("Negative")
    else:
        print("Positive")
predict_sentiment(review)

Negative


In [17]:
joblib.dump(model, "models/sentiment_model.pkl")
print("Model saved as sentiment_model.pkl")

Model saved as sentiment_model.pkl
